In [ ]:
# Import necessary libraries and dependencies
from bs4 import BeautifulSoup
import os
import urllib
import re
import time
import pandas as pd
import json

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
# Set wait times
waittime = 30
sleeptime = 0.5

In [ ]:
# Initiate web driver
try:
    driver.close()  # Close any existing WebDrivers
except Exception:
    pass

In [ ]:
# Set webdriver options
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('ignore-certificate-errors')

In [ ]:
# Specify the path to Chrome WebDriver executable depending on where your chromedriver is stored
webdriver_path = r'C:\WebDriver\chromedriver.exe'

In [ ]:
# Initiate webdriver with the specified path
driver = webdriver.Chrome(executable_path=webdriver_path, options=options)

In [ ]:
# Define target website. We are using 99.co's listing of condominiums. Page was last checked on 6 Jun
home_page = "https://www.99.co/singapore/condos-apartments?alphabet=a"

In [ ]:
# Get driver to retrieve homepage
driver.get(home_page)

In [ ]:
# Sleep a short while for page loading to be fully completed
time.sleep(sleeptime)

In [ ]:
# # Parse and display HTML of results page 1, to take a look
results_html = BeautifulSoup(driver.page_source, 'html.parser')
results_html.head()

(1) Test the code on the first page before scraping all pages 

In [ ]:
pty_name = driver.find_elements(By.XPATH, "//div[@class='_2IBku _32zfS']")
pty_name_list = []
for i in pty_name:
    pty_name_list.append(i.text)

In [ ]:
# Create a DataFrame with the scraped data
data = pd.DataFrame({'name': pty_name_list})

In [ ]:
# Save the DataFrame to a CSV file
file_name = 'condonames.csv'
data.to_csv(file_name, index=False)

In [ ]:
print(f"Scraped data saved to {file_name}.")

(2) Create fucnction to get current page number

In [ ]:
def get_current_page():
    WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, "//li[@class='isActive']/a")))

    current_page_element = driver.find_element(By.XPATH, "//li[@class='isActive']/a")
    current_page = current_page_element.text

    return current_page

In [ ]:
get_current_page()

(3)  Create fucnction to get absolute last page number which is last element before the next arrow

In [ ]:
def get_absolute_last_page():
    WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, "//li/a[@aria-label]")))

    page_elements = driver.find_elements(By.XPATH, "//li/a[@aria-label]")
    last_page_element = page_elements[-1]
    last_page_number = last_page_element.text

    return last_page_number

In [ ]:
get_absolute_last_page()

In [ ]:
# (4) Create scraper function
def scrape_pages():
    current_page = int(get_current_page())
    last_page = int(get_absolute_last_page())
    alphabet = 'a'

    pty_name_list = []

    while True:
        try:
            pty_name = driver.find_elements(By.XPATH, "//div[@class='_2IBku _32zfS']")

            for i in pty_name:
                pty_name_list.append(i.text)

            if current_page == last_page:
                if alphabet == 'x':
                    alphabet = chr(ord(alphabet) + 1)  # Assuming 'x' has no results, skip 'x' and move to the next alphabet
                elif alphabet == 'z':
                    alphabet = '%23'  # Set alphabet to '%23' after 'z'
                else:
                    alphabet = chr(ord(alphabet) + 1)  # Increment the alphabet
                next_page = 1  # Reset the page number
            else:
                next_page = current_page + 1  # Go to the next page

            next_url = f"https://www.99.co/singapore/condos-apartments?alphabet={alphabet}&page={next_page}"
            driver.get(next_url)

            # Update the current page and last page for the new alphabet
            current_page = int(get_current_page())
            last_page = int(get_absolute_last_page())

        except Exception as e:
            print(f"An error occurred: {e}")
            break  # Exit the loop if an error occurs

    # Create a DataFrame with the scraped data
    data = pd.DataFrame({'Name': pty_name_list})

    # Save the DataFrame to a CSV file
    file_name = 'condonames.csv'
    data.to_csv(file_name, index=False)

    print(f"Scraped data saved to {file_name}.")

In [ ]:
# Call the scrape_pages() function
scrape_pages()

In [ ]:
# Display time concluded
print(time.strftime("%H:%M:%S", time.localtime()))